In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
ticker = yf.Ticker("AAPL")
df = ticker.history(start="2022-01-01", end="2024-12-31", interval="1d")
df = df.dropna()

def bollinger_bands(df, window=20, num_std=2):
    df['BB_MA'] = df['Close'].rolling(window).mean()
    df['BB_STD'] = df['Close'].rolling(window).std()
    df['upper'] = df['BB_MA'] + num_std * df['BB_STD']
    df['lower'] = df['BB_MA'] - num_std * df['BB_STD']
    return df

def macd(df):
    ema12 = df['Close'].ewm(span=12, adjust=False).mean()
    ema26 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema12 - ema26
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    return df


def compute_rsi(data, period=14):
    delta = data['Close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    rs = avg_gain / (avg_loss + 1e-10)
    rsi = 100 - (100 / (1 + rs))
    data['RSI'] = rsi
    return data

df = compute_rsi(df)
df.dropna(inplace=True)
initial_capital = 1000
capital = initial_capital
in_position = False
entry_price = 0
portfolio = []
stop_loss = 0
take_profit = 0
trades = []
drawdowns = []
max_portfolio = initial_capital

for i in range(len(df)):
    rsi = df.iloc[i]['RSI']
    close = df.iloc[i]['Close']
    date = df.index[i]

    if not in_position:
        if rsi < 30:
            in_position = True
            entry_price = close
            quantity = capital / close
            stop_loss = close * 0.95
            take_profit = close * 1.10
            entry_date = date
    else:
        stop_loss = max(stop_loss, close * 0.95)
        if close <= stop_loss or close >= take_profit or rsi > 70:
            exit_price = close
            capital = quantity * exit_price
            trades.append({
                'Entry Date': entry_date,
                'Exit Date': date,
                'Entry Price': entry_price,
                'Exit Price': exit_price,
                'Profit': capital - initial_capital
            })
            in_position = False
            max_portfolio = max(max_portfolio, capital)
            drawdown = (max_portfolio - capital) / max_portfolio
            drawdowns.append(drawdown)
    portfolio.append(capital)
portfolio_series = pd.Series(portfolio, index=df.index[:len(portfolio)])
returns = portfolio_series.pct_change().dropna()
sharpe_ratio = (returns.mean() / returns.std()) * np.sqrt(252) if not returns.empty else 0
max_drawdown = max(drawdowns) if drawdowns else 0
total_trades = len(trades)
winning_trades = sum(t['Profit'] > 0 for t in trades)
losing_trades = total_trades - winning_trades
final_value = portfolio[-1]
print("Initial Capital: ₹1000")
print(f"Final Portfolio Value: ₹{final_value:.2f}")
print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown:.2%}")
print(f"Total Trades: {total_trades}")
print(f"Winning Trades: {winning_trades}")
print(f"Losing Trades: {losing_trades}")



Initial Capital: ₹1000
Final Portfolio Value: ₹1526.41
Sharpe Ratio: 0.95
Max Drawdown: 10.11%
Total Trades: 22
Winning Trades: 20
Losing Trades: 2
